In [ ]:
import numpy as np
import scipy as sc
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import sompy

import random

import os
import sys
from collections import Counter
from collections import namedtuple

exploration
->
dataprep
->
clustering
->
classification


* feature combination
* data imputance
* dimension reduction and outlier removal
* re-balancing using SMOTE
* add F1-score minimizer, see [this](https://towardsdatascience.com/the-unknown-benefits-of-using-a-soft-f1-loss-in-classification-systems-753902c0105d) blog

Model
* dummy classifiers
* ensemble of classifiers
* hyperoptimisation

Models:
* ensemble trees: XGB, LGBM
* probabilistic: NB, GPC
* linear: GAM, LR, Huber, Ridge, Lasso, LDA
* kernel: nuSVM
* neural: MLP
* other: quadratic classifier, [learning vector quantization 1](https://machinelearningmastery.com/implement-learning-vector-quantization-scratch-python/), [learning vector quantization 2](http://neupy.com/docs/tutorials.html)

Compare removing outliers from test to increasing threshold.